#### Code Outline:

This code is for contour extraction from pots with with amphora-like handles, i.e. handles that have been stuck on, that stick back on to the pot.

The main difference in this contour extraction code is that unlike the others where we only found one contour showing the outline of the pot, here we must also find the two contours depicting the handles.

At present, this extraction code is for all pots with amphora-like handles. In future, it could be possible to add filters so different ranges can be used to find the handle contours, depending on where the handles are located on the pot. 

Note that here we try binarize all photos and then find the contours. In theory, a perhaps a .txt or .csv file should be made with labels such as "dark" or "not_dark" for each pot, so that we only binarize pot images that aren't too dark. This is because there are times when the background is so dark, that the binarize image barely depicts the pot. Also, at the moment, for those that cannot physically be binarized, we use the original image.

Out of all pot types, these pots are probably the most important ones to binarize. This is because we are looking for three contours, and often contours found from paintings in the middle, get wrongly picked to be a handle contours.

Lastly, though this algorithm works for extracting contours of the outline and handles of pots with handles that go over the top, e.g. nestoris pots, it still requires an extra step to be made in order to remove these handles.

In [1]:
import matplotlib.pyplot as plt 
import numpy as np
import pylab as pl
import pandas as pd
from math import sqrt
from skimage.filters import threshold_otsu
from skimage import measure
import os
import re
import itertools as it
import csv
from PIL import Image
from skimage.color import rgb2gray
from skimage.filters import gaussian
from skimage.segmentation import active_contour
from skimage import data, img_as_float
from skimage.segmentation import (morphological_chan_vese,
                                  morphological_geodesic_active_contour,
                                  inverse_gaussian_gradient,
                                  checkerboard_level_set)
pl.ion()

### Functions

In [540]:
def find_centre(x,y):
    yc = min(y)+(max(y)-min(y))/2
    tnth = max(y)-(max(y)-min(y))/10
    coords = np.where((np.array(y)<tnth) & (np.array(y)>yc)) [0]
    xc = min(np.array(x)[coords])+(max(np.array(x)[coords])-min(np.array(x)[coords]))/2
    return xc,yc,coords

In [545]:
def get_contour_side(X,Y,D):
    
    xc,yc,coords = find_centre(X,Y)    
    
    tnth = min(X) + (max(X)-min(X))/10
    lb = xc - tnth
    ub = xc + tnth
    rng = []
    
    rng = np.where((X >= lb) & (X <=ub))[0]
    
    ktop = np.argmax(Y[rng])
    top_pnt = rng[ktop]
    
    kbot = np.argmin(Y[rng])
    bot_pnt = rng[kbot]

    mx_pnt = max(top_pnt,bot_pnt)
    mn_pnt = min(top_pnt,bot_pnt)
        
    xs1 = list(X[mn_pnt:mx_pnt])
    ys1 = list(Y[mn_pnt:mx_pnt])

    xs2 = list(X)[mx_pnt:] + list(X)[:mn_pnt] 
    ys2 = list(Y)[mx_pnt:] + list(Y)[:mn_pnt] 
    
    if D == "R":
        xs = xs2
        ys = ys2
    else:
        if D == "L":
            xs = xs1
            ys = ys1
        else:
            if min(xs1) > min(xs2):
                D1 = "R"
                D2 = "L"
            else:
                D1 = "L"
                D2 = "R"

            if len(ys1) >= len(ys2):
                ys = ys2
                xs = xs2
                D = D2
            else:
                ys = ys1
                xs = xs1
                D = D1


    return xs,ys,xc,yc,D

In [560]:
def edit_pot_ends(x,y,xs,ys,side):
    
    xc,yc,coords = find_centre(x,y)
    
    if side == "unknown":
        if min(np.array(x)[coords]) < xc:
            side = "L"
        else:
            side = "R"
    
    if side == "L":
        coords2 = np.where(np.array(xs)<xc)[0]
    else:
        coords2 = np.where(np.array(xs)>xc)[0]
        
    xnew = []
    ynew = []
    xnew.append(xc)
    ynew.append(ys[0])
    xnew.extend(np.array(xs)[coords2])
    ynew.extend(np.array(ys)[coords2])
    xnew.append(xc)
    ynew.append(ys[-1])
   
    
    if side == "L":
        xnew = [(-1*i)+2*xc for i in xnew]

    return xnew,ynew

In [535]:
def get_top_bot_handle(x_,y_):

    mx = np.argmax(y_)
    mn = np.argmin(y_)
    ubt = np.max(y_)
    lbb = np.min(y_)
    ubb = ubt - ((ubt - lbb)/10) #used to be /10
    lbt = lbb + ((ubt - lbb)/10)

    bot = []
    top = []

    for i in range(0,len(x_)):
        if lbb <= y_[i] <= lbt:
            bot.append(i)
        if ubb <= y_[i] <= ubt:
            top.append(i)
    return top,bot

In [8]:
def get_handle_side(x_,y_,top,bot,D):
    if D == "L":
        p1 = np.argmin(x_[top])
        p2 = np.argmax(x_[bot])
    else:
        p1 = np.argmax(x_[top])
        p2 = np.argmin(x_[bot])

    p1 = top[p1]
    p2 = bot[p2]

    # Determine direction of contour.
    d_ = "L"
    if (x_[p2] > x_[p2-1]) or (x_[p2] > x_[p2-2]):
        d_ = "R"

    xh = []
    yh = []

    x_2 = list(x_)
    y_2 = list(y_)

    # Case 1:
    if (d_ == "R" and D == "L") or (d_ =="L" and D == "R"):
        if p1 > p2:
            xh = x_2[p2:p1]
            yh = y_2[p2:p1]
        else:
            xh = x_2[p2:]
            xh.extend(x_2[0:p1])
            yh = y_2[p2:]
            yh.extend(y_2[0:p1])
    # Case 2:        
    elif (d_ == "L" and D == "L") or (d_ =="R" and D == "R"):
        if p1 > p2:
            xh = x_2[p1:]
            xh.extend(x_2[0:p2])
            yh = y_2[p1:]
            yh.extend(y_2[0:p2])
        else:
            xh = x_2[p1:p2]
            yh = y_2[p1:p2]
    
    return xh,yh

In [13]:
def smooth_side_contour(x,y,direction):
    
    ys = sorted(y)
    ys = np.unique(np.round(ys))
    y = np.round(y)
    x = np.round(x)
        
    xs = []
    for i in range(0,len(ys)):
        inds = np.where(y==ys[i])
        xy = x[inds]
        if direction == 'R':
            xs.append(max(xy))
        else:
            xs.append(min(xy))
            
    return xs,ys

In [20]:
def binary_coloured(img):
    x_len = len(img[0])
    y_len = len(img)

    x = []
    y = []

    # Find coordinates of red parts (contour)
    for i in range(0,y_len):
        for j in range(0,x_len):
            if (img[i][j][0] < 155) and (img[i][j][1] < 155) and (img[i][j][2] < 155):
                img[i][j] = [0,0,0]
                y.append(i)
                x.append(j)
            else:
                img[i][j] = [255,255,255]
    return img

In [426]:
def get_longest_contours(img):
    
    thresh = threshold_otsu(img)
    binary = img > thresh
    cont = measure.find_contours(binary, 0.8)

    # Find longest contour:
    cont_ln = []
    for n, contour in enumerate(cont):
        cont_ln.append(len(contour))

    longest_c = sorted(cont_ln,reverse=True)[:10]
    long_ind = []
    long_ind_rngsx = []
    long_ind_rngsy = []
    long_ind_mx = []
    long_ind_mn = []


    for i in range(0,len(cont_ln)):
        if cont_ln[i] in longest_c:
            long_ind.append(i)
            c = cont[i]
            x = c[:,1]
            y = c[:,0]
            rngy = abs(max(y) - min(y))
            rngx = abs(max(x) - min(x))
            long_ind_rngsy.append(rngy)
            long_ind_rngsx.append(rngx)
            long_ind_mx.append(max(y))
            long_ind_mn.append(min(y))

    n = len(long_ind)
    
    mn = min(long_ind_mn)
    
    mx = max(long_ind_rngsy)
        
    k1 = np.argmax(long_ind_rngsy)
    
    ub = long_ind_mn[k1] + (mx/4)
    
    inds = list(range(0,n))
    inds.remove(k1)    
    k = (inds[0],inds[1])
    tot = 1000
    
    rng_inds = []
    for i in range(0,n):
        if all([long_ind_rngsy[i] > mx/8,long_ind_mn[i] < ub,long_ind_mx[i] > mn, i!=k1]):
            rng_inds.append(i)
            
    if len(rng_inds) >=2:
        t = 8
    else:
        t = 9
            

    for i in range(0,n):
        if all([long_ind_rngsy[i] > mx/t,long_ind_mn[i] < ub,long_ind_mx[i] > mn, i!=k1]):
            for j in range(i,n):
                if all([long_ind_rngsy[j] > mx/t, long_ind_mn[j] < ub, long_ind_mn[j] > mn, i!=j, j!=k1]):
                    d1 = abs(long_ind_rngsy[i] - long_ind_rngsy[j])
                    d2 = abs(long_ind_rngsx[i] - long_ind_rngsx[j])
                    d3 = abs(long_ind_mx[i] - long_ind_mx[j])
                    sm = d1+d2+d3
                    if sm < tot:
                        tot = sm
                        k = (i,j)


    
    k1 = long_ind[k1]
    k2 = long_ind[k[0]]
    k3 = long_ind[k[1]]


    c1 = cont[k1]
    x1 = c1[:,1]
    y1 = c1[:,0]

    c2 = cont[k2]
    x2 = c2[:,1]
    y2 = c2[:,0]

    c3 = cont[k3]
    x3 = c3[:,1]
    y3 = c3[:,0]
    
    return x1,y1,x2,y2,x3,y3



In [570]:
def find_top_of_handle(xs,ys,yh,yhs,D):
    lb = min(yh) - (max(yhs) - min(yhs))/5
    try:
        rng = np.where(np.array(ys)<lb)[0]
        k = 0
        if D == "L":
            for i in range(1,len(rng)-1):
                if (np.array(xs)[rng[i]] > np.array(xs)[rng[i-1]]) & (np.array(xs)[rng[i]] >= np.array(xs)[rng[i+1]]):
                    k = i
        else:
            for i in range(1,len(rng)-1):
                if (np.array(xs)[rng[i]] < np.array(xs)[rng[i-1]]) & (np.array(xs)[rng[i]] <= np.array(xs)[rng[i+1]]):
                    k = i
        k = rng[k]
    except:
        k = np.argmin(ys)
        
    return k

### Extract Contours

In [569]:
directory = os.fsencode('C:\\Users\\arian\\OneDrive\\Pictures\\pots\\amph_hnd')

direc_fold = 'C:\\Users\\arian\\OneDrive\\Pictures\\pots\\amph_hnd'

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    nme = str(filename)[:-4]
    
    print("\nExtracting from pot: "+nme)

    
    # 2) To get a better contour, we adjust the photo to make it B/W, and then get the longest contour of this.
    # Adjusting the photo like this does not always work therefore we add a try & except.
    
    try:
        # 2.1) Get B/W version of original image.
        image_col = data.load(direc_fold+'\\'+str(filename),as_gray=False)
        image_bw = binary_coloured(image_col)
        # 2.2) Create a plot and temporarily save. This will be the new B/W of the pot.
        fig, ax = plt.subplots(figsize=(4, 6))
        ax.imshow(image_bw)
        ax.set_axis_off()
        plt.savefig("pot_new.jpg")
        plt.close()
        
        # 2.3) Get longest contour from new pot image.
        image_grey = data.load('C:\\Users\\arian\\Documents\\GitHub\\pots\\Code\\pot_new.jpg',as_gray=True)        

    except:
        # 1) Find longest contour from oringal pot image. We assume/hope this is the outline of the pot.
        image_grey = data.load(direc_fold+'\\'+str(filename),as_gray=True)

    x1,y1,x2,y2,x3,y3 = get_longest_contours(image_grey)

    # 3) Get one side of contour. 
    # "L" in the third parameter gets the left side of the pot, "R" the right side, and anything else gets
    # the shortest side.
    xs,ys,xc,yc,D = get_contour_side(x1,y1,"Z")
    
    # 4) Smooth the contour
    xsmooth,ysmooth = smooth_side_contour(xs,ys,D)
    
    # 5) Edit contour ends so that they start and end at the centre of the pot.
    #x,y = edit_pot_ends(x1,y1,xsmooth,ysmooth,D)
    
    if min(x2) < min(x3):
        if D == "L":
            xh = x2
            yh = y2
        else:
            xh = x3
            yh = y3
    else:
        if D == "L":
            xh = x3
            yh = y3
        else:
            xh = x2
            yh = y2
        
    top,bot = get_top_bot_handle(xh,yh)
    xhs,yhs = get_handle_side(xh,yh,top,bot,D)
    
    k1 = find_top_of_handle(xsmooth,ysmooth,yh,yhs,D)
    
    ub = max(yh) + (max(yhs) - min(yhs))/4
    rng2 = np.where(np.array(ysmooth)>ub)[0]
    k2 = rng2[0]
    
    p1 = min(k1,k2)
    p2 = max(k1,k2)
    
    if yhs[0] > yhs[-1]:
        yhs.reverse()
        xhs.reverse()
        
    k = np.argmax(yhs)
    
    xr = []
    yr = []
    if p1 == 0:
        xr.append(xsmooth[p1])
        yr.append(ysmooth[p1])
        xr.extend(xhs[1:k])
        yr.extend(yhs[1:k])
    else:
        xr.extend(xsmooth[:p1])
        yr.extend(ysmooth[:p1])
        xr.extend(xhs[2:k])
        yr.extend(yhs[2:k])
        
    xr.extend(xsmooth[p2:])
    yr.extend(ysmooth[p2:])

        
    x,y = edit_pot_ends(x1,y1,xr,yr,D)
    
    fig, ax = plt.subplots(figsize=(4, 6))
    ax.imshow(image_grey,cmap='gray')
    ax.set_axis_off()
    ax.plot(x,y,'--y')
    plt.savefig(nme+"_hnd_rmv_test3.png")
    plt.close()

    


Extracting from pot: amphiskoros_1003839

Extracting from pot: amphiskoros_1010424

Extracting from pot: amphiskoros_9031756

Extracting from pot: amphora_9003875

Extracting from pot: amphora_9007230

Extracting from pot: amphora_a_7129

Extracting from pot: amphora_b_6154

Extracting from pot: amphora_neck12972

Extracting from pot: amphora_neck_4529

Extracting from pot: amphora_panathenic_10729

Extracting from pot: amphora_panathenic_512

Extracting from pot: amphora_panathenic_9024203

Extracting from pot: colkrater_8937_BM.

Extracting from pot: cup_kantharos_45406

Extracting from pot: cup_kantharos_9017310

Extracting from pot: cup_kantharos_9033465

Extracting from pot: cup_mastoid_310505

Extracting from pot: cup_skyphos_9033374

Extracting from pot: krater_9037476

Extracting from pot: krater_volute_1006971

Extracting from pot: krater_volute_9989

Extracting from pot: nestoris_9004535

Extracting from pot: pelike_202000
